# Generate lanHEP input in the mass eigenstate basis. 

In [1]:
import Pseudo_Goldstone as pg
import os
import sympy as sp
import numpy as np

In [2]:
DimN=2
RunParallel=True
testit=1 #test diagonalization of Mhro. Change it to 0 if it is slow.

#the lanHEP mdl files will be here:
dirV='lanHEP/mdl_files'
if not os.path.exists(dirV):
    os.makedirs(dirV)

dirV=dirV+"/SU"+ str(DimN)
if not os.path.exists(dirV):

    os.makedirs(dirV ) 

lanHep_file=open(dirV+'/SU'+ str(DimN)+'_pseudoGoldstone.mdl','w')

file_misc=open('lanHEP/misc/SM1.mdl','r')
lanHep_file.write(file_misc.read())
file_misc.close()

for tmpgauge in ['feyn','un']:
    print tmpgauge
    
    Fields ,ParameterSymbol=pg.Definitions(DimN,tmpgauge)

    LMassInt=pg.GetLagrangian(Fields)
    Point_N=pg.IdentifyInteractions(LMassInt,Fields ,Parallel=RunParallel)
    pg.Make_Feynman_Rules(Point_N)
    pg.CheckInteractions(Point_N,LMassInt,Fields)
    pg.StoreVert(Point_N,Fields,ParameterSymbol,DimN,tmpgauge,Directory='Frules')

    
    
    [sp.var(str(i)) for i in Fields];
    # h rho mass matrix
    

    Fhrho=[h,rho] 
    
    tmphrho=sp.Symbol('tmphrho')
    Mhrho=[ [   pg.VertexValue(i,j) if j!=Fhrho[1] or i!=Fhrho[1] else tmphrho   \
             for j in  Fhrho ] for i in Fhrho  ]#in order to speed things up, I use tmphrgo for Mhrho[1][1].
                                                #this is restored later (see after the 
                                                # check for Fhrho.Mhrho.Fhrho-(H**2*mh12 + S**2*mh22)),
                                                # since it affects only the mh12 and mh22!
    SubsTmp={tmphrho:pg.VertexValue(Fhrho[1],Fhrho[1])}

    MH , MS0=sp.symbols('MH MS0')
    th=sp.Symbol('th')
    
    FHS=[sp.Symbol('H'),sp.Symbol('S0')]
    Ohrho=1/sp.sqrt(1+th**2)*np.array([[1,-th],[th,1]])

    Oh11, Oh12=sp.symbols('Oh11 Oh12')

    Oh=np.array([[Oh11,-Oh12],[Oh12,Oh11]])
    SubsOhTan=[(Oh11,1/sp.sqrt(1+th**2) ),(Oh12,th/sp.sqrt(1+th**2))]

    mh12=(Mhrho[0][0] + Mhrho[1][1]-sp.sqrt(4* Mhrho[0][1]**2 + (Mhrho[0][0] - Mhrho[1][1])**2))/2 
    mh22=(Mhrho[0][0] + Mhrho[1][1]+sp.sqrt(4* Mhrho[0][1]**2 + (Mhrho[0][0] - Mhrho[1][1])**2))/2 


    tmpS=pg.MatrixProd([Oh,FHS])
    SubsDiaghrho=[(Fhrho[i], tmpS[i]) for i in range(len(Fhrho))]
    SubsTanh=[(th, (MH**2 - Mhrho[0][0])/Mhrho[0][1])]

    
    
    #check if Fhrho.Mhrho.Fhrho-(H**2*mh12 + S**2*mh22)==0
    if testit==1:
        
        print 'Mhrho Diagonalization correct?'
        print sp.simplify(sp.expand(\
        pg.MatrixProd([Fhrho,Mhrho,Fhrho]).subs(SubsDiaghrho).subs(SubsOhTan).subs(SubsTanh).subs(MH**2,mh12)\
        -(FHS[0]**2*mh12 + FHS[1]**2*mh22)))==0

    #restore Mhrho[1][1]
    mh12=mh12.subs(SubsTmp)
    mh22=mh22.subs(SubsTmp)
        
    
    #'parameter' and 'let' for h and rho
    paramLHEPHS=[]
    paramLHEPHS+=['parameter lamHPhi= 1']
    paramLHEPHS+=[ 'parameter v=2*MW/EE*SW']
    paramLHEPHS+=['parameter ' +'mPhi2{}{}= 1000'.format(DimN,DimN)]
    paramLHEPHS+=['parameter ' +'mPhip2{}{}= 1000'.format(DimN,DimN)]
        
    LAMH=sp.solve((mh12+mh22)-(MH**2+MS0**2),sp.Symbol('lamH'))[0]
    VPhi2=sp.solve((mh12-mh22)**2-(MH**2-MS0**2)**2,sp.Symbol('vPhi')**2)[0]
    VPhi=sp.sqrt(VPhi2).subs('lamH',LAMH)
    
    paramLHEPHS+=['parameter ' +'lamH='+str(LAMH)]
    paramLHEPHS+=['parameter ' +'vPhi='+str(VPhi).replace('Abs','fabs')]    
        
    paramLHEPHS+= ['parameter ' + str(i[0])+'='+str(i[1]) for i in SubsTanh+SubsOhTan]



    letLHEPHS= [ 'let ' + str(i[0])+'='+str(i[1]) for i in SubsDiaghrho]






    #From phi,s,chi to G1,G2,G3... (ie diagonal PNGBs)
    
    OG=np.array(sp.Matrix( (2*DimN-1), (2*DimN-1),  lambda i,j: 'OG'+str(i+1)+str(j+1)))
    FG=Fields[:]
    FG.remove(h)
    FG.remove(rho)
    if tmpgauge!='un':
        FG.remove(G0)
        FG.remove(Gp)
        FG.remove(Gm)
    

    FGDiag=[ sp.Symbol('G'+str(i)) for i in range(1,len(FG)+1 )]
    tmpS=pg.MatrixProd([OG,FGDiag])
    SubsDiagG=[(FG[i], tmpS[i]) for i in range(len(FG))]


    paramLHEPGoldstone= [ 'parameter ' + str(i)+'=1'for i in sp.flatten(OG)]
    letLHEPGoldstone= [ 'let ' + str(i[0])+'='+str(i[1]) for i in SubsDiagG]



    FAll=np.append(FHS,FGDiag)



    ###Print the lanHEP input

    if tmpgauge=='feyn':
        lanHep_file.write( 'do_if gauge_fixing==Feynman.')
        lanHep_file.write('\n')

    else:
        lanHep_file.write( 'do_else.')
        lanHep_file.write('\n')
    
        
    lanHep_file.write(     '%%%%%%%%%%%%%%%%%%%%%% Definition of particles:')
    lanHep_file.write('\n')

    pdg=10000
    for i in FAll:
        if i==sp.Symbol('H'):
            lanHep_file.write( 'scalar '+str(i)+'/'+str(i)+' :(\'H-boson\' , mass MH=125, width wH=auto ).')
            lanHep_file.write('\n')
        else:
            lanHep_file.write('scalar '+str(i)+'/'+str(i)+' :( \''+str(i)+ '-boson\' , \
            mass M'+str(i) +'=1000, width w'+str(i)+'=auto , pdg '+ str(pdg)+ ').')
            lanHep_file.write('\n')
            pdg+=1
    lanHep_file.write('\n')
    lanHep_file.write(     '%%%%%%%%%%%%%%%%%%%%%% Definition of parameters:')
    lanHep_file.write('\n' )      
    #params1= 'parameter  lamPhi=1 , vPhi=1000 , v=2*MW/EE*SW.'
    #lanHep_file.write( params1)
    lanHep_file.write('\n')


    for i in paramLHEPGoldstone+paramLHEPHS+letLHEPGoldstone+letLHEPHS:
        lanHep_file.write( str(i)+'.')
        lanHep_file.write('\n')

        

    if tmpgauge=='feyn':
        lanHep_file.write('\n')
        lanHep_file.write( 'let pp = { -i*\'W+.f\',  (vev(2*MW/EE*SW)+h+i*\'Z.f\')/Sqrt2 }, ')
        lanHep_file.write( ' PP = {  i*\'W-.f\',  (vev(2*MW/EE*SW)+h-i*\'Z.f\')/Sqrt2 }.    ')
        lanHep_file.write('\n')
    else:
        lanHep_file.write('\n')
        lanHep_file.write( 'let pp = {0,  (vev(2*MW/EE*SW)+h)/Sqrt2 }, ')
        lanHep_file.write( ' PP = { 0,  (vev(2*MW/EE*SW)+h)/Sqrt2 }.    ')
        lanHep_file.write('\n')

        
        
    lanHep_file.write('\n')
    lanHep_file.write(     '%%%%%%%%%%%%%%%%%%%%%% Lagrangian terms:')
    lanHep_file.write('\n')



    GaugeSubs=[  (G0,sp.Symbol('Z.f')),\
                 (Gp,-sp.I*sp.Symbol('W+.f')),\
                 (Gm,sp.I*sp.Symbol('W-.f'))\
              ]

    SubsCust=[(sp.sqrt(2),sp.Symbol('Sqrt2')),\
              (sp.I,sp.Symbol('i')), \
              (sp.Symbol('Z.f'),' \'Z.f\''  ), \
              (sp.Symbol('W+.f'),'\'W+.f\''  ), (sp.Symbol('W-.f'),'\'W-.f\''  )]
    
    for i in Point_N[3]:
        lanHep_file.write( 'lterm '+str(((np.product(i[0])*i[1]).subs(GaugeSubs)).subs(SubsCust)  ) +'.'   )
        lanHep_file.write('\n')

    for i in Point_N[4]:
        lanHep_file.write( 'lterm '+str(((np.product(i[0])*i[1]).subs(GaugeSubs)).subs(SubsCust)  ) +'.'   )
        lanHep_file.write('\n')

        
    if tmpgauge!='feyn':
        lanHep_file.write('end_if. \n')

file_misc=open('lanHEP/misc/SM2.mdl','r')
lanHep_file.write(file_misc.read())
file_misc.close()  



  
lanHep_file.close()       

feyn
Checking vanishing of the first derivatives of the potential...
The conditions are correct!
calculating 2-point interactions
calculating 3-point interactions
calculating 4-point interactions
Checking Vertices...
The interactions have been identified correctly!!
Writing Vertices (Feynman Rules and mass matrix entries)...
All Done!
Mhrho Diagonalization correct?
True
un
Checking vanishing of the first derivatives of the potential...
The conditions are correct!
calculating 2-point interactions
calculating 3-point interactions
calculating 4-point interactions
Checking Vertices...
The interactions have been identified correctly!!
Writing Vertices (Feynman Rules and mass matrix entries)...
All Done!
Mhrho Diagonalization correct?
True


Example for how to produce mdl files for calchep (similarly you can make ufo files).
```
%%bash -s $DimN 

for i in "unitary" "Feynman"
do
DIRECTORY="Frules/SU$1/CalcHEP-$i"
if [ ! -d "$DIRECTORY" ]; then 
mkdir $DIRECTORY
fi


/path/to/lanhep -ca -key gauge_fixing=$i  \
-OutDir Frules/SU$1/"CalcHEP-"$i lanHEP/mdl_files/SU$1/SU$1_pseudoGoldstone.mdl
done
```